<font size="-1">
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <b>3D Object Representations for Fine-Grained Categorization</b><br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei<br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <i>4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013</i> <b>(3dRR-13)</b>. Sydney, Australia. Dec. 8, 2013.<br>
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
            <a style="text-decoration: underline" href="../papers/3drr13.pdf">[pdf]</a>
            &nbsp;&nbsp;<a style="text-decoration: underline" href="../papers/3drr13.bib">[BibTex]</a>
            &nbsp;&nbsp;<a style="text-decoration: underline" href="../papers/3drr_talk.pdf">[slides]</a>
</font>

Pour créer un env dédié:
conda create -n cars numpy=1.19.1 scipy tensorflow keras matplotlib notebook pandas Pillow opencv-python

Avec support GPU:
conda create --name tf_gpu numpy=1.19.1 scipy tensorflow-gpu keras-gpu matplotlib notebook pandas Pillow opencv-python  
Ressources utilisées :  
https://medium.com/swlh/recognising-cats-and-dogs-using-neural-networks-with-tensorflow-6f366ad30dbf  
https://towardsdatascience.com/how-to-load-matlab-mat-files-in-python-1f200e1287b5  
  
    
Besoin de 3 datasets:
- training 
- validation
- test

Tout est fourni dans le dataset  choisi

dataset from https://ai.stanford.edu/~jkrause/cars/car_dataset.html  
NB: Does not feature a lot of french car brands

In [20]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("imports ok. Number of GPUs Available sould be == 1")

Number of GPUs Available:  0
imports ok. Number of GPUs Available sould be == 1


In [21]:
tf.test.is_built_with_cuda()

False

In [2]:
#import des infos sur les classes de voitures
from scipy.io import loadmat
meta_annot = loadmat('car_devkit\cars_meta.mat')
data = [[row.flat[0] for row in line] for line in meta_annot['class_names'][0]]
print(len(data))
columns = ["model_id", "class_name",]
df_names = pd.DataFrame(list(zip(range( 1,len(data)+1), data)), columns=columns)
df_names.head()

196


,model_id,class_name
0,1,[AM General Hummer SUV 2000]
1,2,[Acura RL Sedan 2012]
2,3,[Acura TL Sedan 2012]
3,4,[Acura TL Type-S 2008]
4,5,[Acura TSX Sedan 2012]


In [3]:
#Il faut maintenant regrouper les classes de véhicules par marque au lieu du modèle
df_names["b_name"] = df_names.class_name.astype(str).apply(lambda x: x.split(" ")[0].replace("['",""))
df_names["b_name"] = df_names["b_name"].apply(lambda x: "AM General" if x=="AM" else x)
df_names["b_id"] = pd.factorize(df_names["b_name"])[0] #creation du nouvel index
# set(df_names.cnames.tolist()) #visualisation de la liste des marques
# df_names.b_name.value_counts() #nombre de modèles dans le dataset par marque
# df_names.b_id.value_counts() #nombre de répétitions des identifiants de marque pour validation
df_names

,model_id,class_name,b_name,b_id
0,1,[AM General Hummer SUV 2000],AM General,0
1,2,[Acura RL Sedan 2012],Acura,1
2,3,[Acura TL Sedan 2012],Acura,1
3,4,[Acura TL Type-S 2008],Acura,1
4,5,[Acura TSX Sedan 2012],Acura,1
...,...,...,...,...
191,192,[Volkswagen Beetle Hatchback 2012],Volkswagen,46
192,193,[Volvo C30 Hatchback 2012],Volvo,47
193,194,[Volvo 240 Sedan 1993],Volvo,47
194,195,[Volvo XC90 SUV 2007],Volvo,47


In [4]:
#création d'un dict pour convertir facilement les model_id en b_id
id_conv= dict(zip(df_names["model_id"],df_names["b_id"]))
# pour obtenir le numéro de marque du modèle #44 :
print(df_names.loc[df_names['model_id'] == 44])
print()
print(f"{id_conv[44]=}")

#idem pour les noms de marques:
name_conv= dict(zip(df_names["b_id"],df_names["b_name"]))

    model_id                                    class_name   b_name  b_id
43        44  [Bentley Continental Flying Spur Sedan 2007]  Bentley     5

id_conv[44]=5


In [5]:
#import des annotations de training depuis le fichier matlab
from scipy.io import loadmat

mat_annot = loadmat('car_devkit\cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in mat_annot['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
df_train.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,fname
0,39,116,569,375,14,00001.jpg
1,36,116,868,587,3,00002.jpg
2,85,109,601,381,91,00003.jpg
3,621,393,1484,1096,134,00004.jpg
4,14,36,133,99,106,00005.jpg


In [ ]:
#ajouter les b_id à la dataframe en utilisant le dict et apply:
df_train["b_id"] = df_train["class"].apply(lambda x: id_conv[x])
# reordering the df
df_train = df_train [["bbox_x1", "bbox_y1","bbox_x2","bbox_y2","class", "b_id", "fname"]]
df_train.head()

In [ ]:
#nombre d'images par marque de voiture :
df_train.b_id.apply(lambda x: name_conv[x]).value_counts()

In [ ]:
#import des infos de test 
# sera peut-être utile plus tard :)
from scipy.io import loadmat
test_annot = loadmat('car_devkit\cars_test_annos.mat')
data = [[row.flat[0] for row in line] for line in test_annot['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
df_test = pd.DataFrame(data, columns=columns)
df_test.head()
#we are not sending our results -> this may be useless

In [ ]:
import matplotlib.pyplot as plt

#fonction pour plotter les box sur les images
def draw_rect(row, color = None):
    
    img_f = plt.imread(r"cars_train\\" + row['fname'], format='jpeg')
    
    if not color:
        color = [255,255,255]
        
    pt1, pt2 = (row['bbox_x1'], row['bbox_y1']) , (row['bbox_x2'], row['bbox_y2'])
    pt1 = int(pt1[0]), int(pt1[1])
    pt2 = int(pt2[0]), int(pt2[1])
    img_f = cv2.rectangle(img_f.copy(), pt1, pt2, color, int(max(img_f.shape[:2])/200))
    
    return img_f

# this synthax can be used to draw bounding boxes on the images
plt.imshow(draw_rect(df_train.iloc[686]))

In [ ]:
# ouvrir une image avec numpy:
img = plt.imread(r"cars_train\00001.jpg", format='jpeg')

In [ ]:
import cv2

#fonction pour plotter les box sur les images
def crop_img(row):
    img_f = plt.imread(r"cars_train\\" + row['fname'], format='jpeg')
    # x and y are flipped when slicing 😠 np arrays
    return img_f[row['bbox_y1']:row['bbox_y2'], row['bbox_x1']:row['bbox_x2']]

# this synthax can be used to crop the images
plt.imshow(crop_img(df_train.iloc[686]))


In [ ]:
#splitting the training dataframe into training, validation and test dfs
# litterature suggests :
#  70% of the entire Dataset for training (Training data) : df_flow_train
#  15% of the entire Dataset for validation (Validation data): df_flow_val
#  15% of the entire Dataset for testing (Testing data): df_flow_test
# one issue here : if our sampling is is not representative, we can end up with unbalanced datasets

# creating the empty dfs:
column_names = [col for col in df_train.columns]
df_flow_train  = pd.DataFrame(columns = column_names) 
df_flow_val  = pd.DataFrame(columns = column_names) 
df_flow_test  = pd.DataFrame(columns = column_names) 

# we need to sample x amount for each class instead of for the entire df:

for i in range(df_train['b_id'].max()+1):
    df_flow_train = df_flow_train.append(df_train[df_train['b_id']==i].sample(frac = 0.7))

df_train_rest = df_train.drop(df_flow_train.index) 

for i in range(df_train['b_id'].max()+1):
    df_flow_val = df_flow_val.append(df_train_rest[df_train_rest['b_id']==i].sample(frac = 0.5))

df_flow_test = df_train_rest.drop(df_flow_val.index) 

#checking the datasets len():
print(len(df_flow_train))
print(len(df_flow_val))
print(len(df_flow_test))

#checking if we have duplicates, should be 0
print(len(df_flow_train[df_flow_train.duplicated()]))
print(len(df_flow_val[df_flow_val.duplicated()]))
print(len(df_flow_test[df_flow_test.duplicated()]))

#checking if we have overlap in the datasets, should be 0
remerged = df_flow_train.append(df_flow_val).append(df_flow_test)
print(len(remerged[remerged.duplicated()]))


In [ ]:
# adding a cat column to use for the classifier
df_flow_train["cat_str"] = df_flow_train["b_id"].apply(lambda x: name_conv[x])
df_flow_val["cat_str"] = df_flow_val["b_id"].apply(lambda x: name_conv[x])
df_flow_test["cat_str"] = df_flow_test["b_id"].apply(lambda x: name_conv[x])

df_flow_train.head()

In [ ]:
# danger zone : cropping 7k images at once, what could go wrong lol
# don't overwrite the initial images! Uncomment and remove # in the middle to use
# df_train.apply(#lambda x: cv2.imwrite(r"cars_cropped\\" + x['fname'], crop_img(x)), axis=1)

In [ ]:
img_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.1, shear_range=0.1)
img_size = 200
batch_size = 32

train_img_gen = img_gen.flow_from_dataframe(dataframe= df_flow_train,
                                            directory="cars_cropped\\",
                                            x_col = "fname",
                                            y_col = "cat_str",
                                            batch_size = batch_size,
                                            target_size=(img_size, img_size),
                                            class_mode = "categorical"
                                           )

validate_img_gen = img_gen.flow_from_dataframe(dataframe= df_flow_val,
                                            directory="cars_cropped\\",
                                            x_col = "fname",
                                            y_col = "cat_str",
                                            batch_size = batch_size,
                                            target_size=(img_size, img_size),
                                            class_mode = "categorical"
                                           )

test_img_gen = img_gen.flow_from_dataframe(dataframe= df_flow_test,
                                            directory="cars_cropped\\",
                                            x_col = "fname",
                                            y_col = "cat_str",
                                            batch_size = batch_size,
                                            target_size=(img_size, img_size),
                                            class_mode = "categorical"
                                           )

In [ ]:
def showImages(arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
showImages([train_img_gen[0][0][0] for i in range(15)])

In [ ]:
numclasses = df_train['b_id'].max()+1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(200, 200, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(numclasses, activation='softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', # feed the model to the optimizer
              metrics=['accuracy'])

model.summary()

In [ ]:
#training
train_total = len(df_flow_train)
validate_total = len(df_flow_val)
# batch_size = 32 #defined earlier

fit_result = model.fit(
            train_img_gen,
            steps_per_epoch=int(np.ceil(train_total / float(batch_size))),
            epochs=5, 
            validation_data=validate_img_gen,
            validation_steps=int(np.ceil(validate_total / float(batch_size)))
            )

In [ ]:
print(fit_result.history['accuracy'])
model.save('firstmodel.h5')

Num GPUs Available:  0
